In [1]:
import numpy as np
import pandas as pd
import re
import string
import time
from sklearn.model_selection import train_test_split
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

/home/awmarcel/anaconda3/envs/customer-complaints-classification-env/lib/python3.10/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/awmarcel/anaconda3/envs/customer-complaints-classification-env/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/awmarcel/anaconda3/envs/cus

In [2]:
# Load data:
df = pd.read_csv("../data/processed/labelled_texts.csv")
df = df[['complaint_what_happened', 'topic']]

In [3]:
# Split data:
X = df['complaint_what_happened'].to_numpy()
y = df['topic'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, stratify=y_test, test_size=0.5, random_state=42)

print(f"Train ratio split: {X_train.shape[0]/ len(df):.1%}")
print(f"Validation ratio split: {X_val.shape[0]/ len(df):.1%}")
print(f"Test ratio split: {X_test.shape[0]/ len(df):.1%}")

Train ratio split: 80.0%
Validation ratio split: 10.0%
Test ratio split: 10.0%


# Util Functions for Text Preprocessing

In [4]:
# Functions
def remove_punctuations(text: str) -> str:
    """Remove punctuations from a text.
    
    Args:
        text (str): text.
    Returns:
        str: text with removed punctuations.
    """
    pattern = f'[{re.escape(string.punctuation)}]'
    return re.sub(pattern, ' ', text)

def remove_numbers(text: str) -> str:
    """Remove numbers from a text.
    
    Args:
        text (str): text.
    Returns:
        str: text with numbers punctuations.
    """
    pattern = r'[0-9]'
    return re.sub(pattern, ' ', text)

def remove_confidential_information(text: str) -> str:
    """Remove confidential information from a text.
    
    Args:
        text (str): text.
    Returns:
        str: text with removed confidential information.
    """
    pattern = r'\b[Xx]{1,}\b'
    return re.sub(pattern, ' ', text)

def remove_extra_spaces(text: str) -> str:
    """Remove extra spaces or new lines from a text.
    
    Args:
        text (str): text.
    Returns:
        str: text with removed extra spaces or new lines.
    """
    pattern = r'\s+'
    return re.sub(pattern, ' ', text)

tokenizer = get_tokenizer('basic_english')
stop_words = stopwords.words('english')
def remove_stopwords(text: str) -> str:
    """Remove stop words from text

    Args:
        text (str): text

    Returns:
        str: text with stop words removed
    """
    tokens = tokenizer(text)
    return ' '.join([token for token in tokens if token not in stop_words])

# source: https://www.ibm.com/topics/stemming-lemmatization#:~:text=The%20practical%20distinction%20between%20stemming,be%20found%20in%20the%20dictionary.
def get_wordnet_pos(tag: str) -> str:
    """Return wordnet constant value to do lemmatization based on their input word tag

    Args:
        tag (str): tag name

    Returns:
        str: constant value for wordnet lemmatization
    """
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:         
        return wordnet.NOUN
    
def lemmatize(text: str) -> str:
    """Perform lemmatization using WordNetLemmatizer

    Args:
        tokens (str): _description_

    Returns:
        str: _description_
    """
    tokens = tokenizer(text)
    pos_tags = pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()  
    return ' '.join([lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in pos_tags])

# Set up mapping for label
label_to_ix = {
    'Bank Account services': 0, 
    'Credit card or prepaid card': 1,
    'Mortgage/Loan': 2, 
    'Theft/Dispute Reporting': 3, 
    'Others': 4
}

ix_to_label = {
    0: 'Bank Account services', 
    1: 'Credit card or prepaid card',
    2: 'Mortgage/Loan', 
    3: 'Theft/Dispute Reporting', 
    4: 'Others'
}

def encode_label(label: str):
    return label_to_ix[label]

def encode_text(text: str, word_to_ix: dict):
    return [word_to_ix.get(word, 0) for word in tokenizer(text)]

In [5]:
# Create text preprocessing pipeline
def process_text(text: str) -> str:
    text = text.lower()
    
    # Text cleaning: 
    text = remove_numbers(text)
    text = remove_punctuations(text)
    text = remove_confidential_information(text)
    text = remove_extra_spaces(text)

    # Text transformation:
    text = remove_stopwords(text)
    text = lemmatize(text)
    return text

# Dataset Class

In [6]:
class CustomerComplaintsDataset(Dataset):
    def __init__(self, encoded_texts: list, encoded_labels: list, max_length: int) -> None:
        super().__init__()
        self.encoded_texts = encoded_texts
        self.encoded_labels = encoded_labels
        self.max_length = max_length
        
    def padding(self, tokens: list, max_length: int):
        if len(tokens) < max_length:
            zeros = list(np.zeros(max_length - len(tokens)))
            new = tokens + zeros
        else:
            new = tokens[:max_length]
        features = np.array(new)
        return features 
    
    def __len__(self):
        return len(self.encoded_labels)

    def __getitem__(self, idx):
        encoded_text = self.padding(self.encoded_texts[idx], self.max_length)
        label = self.encoded_labels[idx]
        return encoded_text, label

# Text Preprocessing

In [7]:
# Process the text
processed_texts = list(map(process_text, X_train[:1000]))

# Generate vocabulary
def yield_tokens(texts):
    for text in texts:
        yield text.strip().split()

vocab = build_vocab_from_iterator(yield_tokens(processed_texts), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Calculate maximum length
max_length = -1  
for text in processed_texts:
    max_length = max(max_length, len(tokenizer(text)))

# Encode texts:
encoded_texts = [vocab(tokenizer(text)) for text in processed_texts] 

# Encode labels:
def collate_fn(batch):
    texts, labels = [], []
    for text, label in batch:
        texts.append(text)
        labels.append(label)
    texts, labels = np.array(texts), np.array(labels)
    return torch.LongTensor(texts), torch.LongTensor(labels)

encoded_labels = [encode_label(label) for label in y_train[:1000]]
dataset = CustomerComplaintsDataset(encoded_texts, encoded_labels, max_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Neural Network Class

In [8]:
class Net(nn.Module):
    def __init__(self, vocab_size, embedding_size, num_classes):
        super(Net, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(embedding_size * max_length, num_classes)
        
    def forward(self, text):
        model = torch.nn.Sequential(
            self.embedding, 
            self.flatten, 
            self.fc
        )
        return model(text)

# Train 

In [11]:
from torchmetrics import Accuracy
from torchmetrics import Recall
from torchmetrics import Precision
from torchmetrics import F1Score


multiclass_acc = Accuracy(task="multiclass", num_classes=num_classes, average='macro')
multiclass_recall = Recall(task="multiclass", num_classes=num_classes, average='macro')
multiclass_precision = Precision(task="multiclass", num_classes=num_classes, average='macro')
multiclass_f1_score = F1Score(task="multiclass", num_classes=num_classes, average='macro')
INTERVAL = 5

def train(model, dataloader, criterion, optimizer):
    # Change model to training mode
    model.train()
    
    total_count = 0
    total_acc = 0
    total_recall = 0
    total_precision = 0
    total_f1_score = 0
    
    for idx, (texts, labels) in enumerate(dataloader):
        # Reset gradients:
        optimizer.zero_grad()

        # Produce model output
        predicted_labels = model(texts)
        
        # Calculate loss
        loss = criterion(predicted_labels, labels)

        # Back Propagation
        loss.backward()
        
        # Update model parameters
        optimizer.step()

        # Evaluate performance
        acc = multiclass_acc(predicted_labels.argmax(1), labels)
        recall = multiclass_recall(predicted_labels.argmax(1), labels)
        precision = multiclass_precision(predicted_labels.argmax(1), labels)
        f1_score = multiclass_f1_score(predicted_labels.argmax(1), labels)
        
        total_acc += acc.item()
        total_recall += recall.item()
        total_precision += precision.item()
        total_f1_score += f1_score.item()
        
        # Count batch
        total_count += 1
        
        # Display model training status
        if idx % INTERVAL  == 0 and idx > 0:
            print(
                "| {:5d}/{:5d} batches | accuracy {:.3f} | recall {:.3f} | precision {:.3f} | f1 score {:.3f}".format(
                    idx, len(dataloader), 
                    total_acc/ total_count, 
                    total_recall / total_count, 
                    total_precision / total_count, 
                    total_f1_score / total_count, 
                )
            )
            total_count = 0
            total_acc = 0
            total_recall = 0
            total_precision = 0
            total_f1_score = 0

In [12]:
# Prepare model hyperparameters
vocab_size = len(vocab) + 1
embedding_size = 128
num_classes = len(df['topic'].unique())

# Create model
model = Net(vocab_size, embedding_size, num_classes)

# Train
lr = 1e-3
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloader, criterion, optimizer)

|     5/   32 batches | accuracy 0.187 | recall 0.187 | precision 0.056 | f1 score 0.081
|    10/   32 batches | accuracy 0.205 | recall 0.205 | precision 0.039 | f1 score 0.064
|    15/   32 batches | accuracy 0.200 | recall 0.200 | precision 0.045 | f1 score 0.073
|    20/   32 batches | accuracy 0.230 | recall 0.230 | precision 0.078 | f1 score 0.107
|    25/   32 batches | accuracy 0.207 | recall 0.207 | precision 0.075 | f1 score 0.095
|    30/   32 batches | accuracy 0.236 | recall 0.236 | precision 0.181 | f1 score 0.144
